In [1]:
import pandas as pd
import numpy as np
from fancyimpute import IterativeImputer

#### 1. Lấy dữ liệu từ data gốc đã crawl được, và xóa đi cột index thừa

In [2]:
data_final = pd.read_csv("data_merge.csv")
del data_final['Unnamed: 0']
data_final

,ID,Location,Area,Bedroom,Bathroom,Frontage,Floors,Price
0,268184,"Quận 12, TP Hồ Chí Minh",130,7.0,6.0,0,3.0,9.2 tỷ
1,267895,"Quận 6, TP Hồ Chí Minh",189,NaN,NaN,1,NaN,24 tỷ
2,268639,"Gò Vấp, TP Hồ Chí Minh",52,4.0,5.0,0,6.0,9.8 tỷ
3,268638,"Gò Vấp, TP Hồ Chí Minh",105,5.0,2.0,1,4.0,8 tỷ
4,268637,"Sơn Trà, Đà Nẵng",115,3.0,2.0,1,2.0,6.2 tỷ
...,...,...,...,...,...,...,...,...
9671,250238,"Hoàng Mai, Hà Nội",54,5.0,3.0,0,4.0,4.95 tỷ
9672,250237,"Đống Đa, Hà Nội",50,4.0,NaN,0,5.0,7.5 tỷ
9673,250235,"Thủ Đức, TP Hồ Chí Minh",100,NaN,NaN,0,NaN,6 tỷ
9674,249841,"Quận 12, TP Hồ Chí Minh",52,3.0,2.0,0,2.0,3.48 tỷ


#### 2. Xử lý những observation mà ở đó có area hay location không tồn tại

- Thay những khu vực có Area không tồn tại (Area is not available) bằng NaN

In [3]:
data_final.loc[data_final['Area'] == 'Area is not available', 'Area'] = np.nan
data_final['Area'] = pd.to_numeric(data_final['Area'], errors='coerce')

- Bỏ luôn những observation không có location

In [4]:
data_final = data_final.drop(data_final[data_final['Location'] == 'Location is not available'].index)
data_final = data_final.reset_index(drop=True)

#### 3. Exploring data analysist

In [5]:
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9671 entries, 0 to 9670
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        9671 non-null   object 
 1   Location  9671 non-null   object 
 2   Area      9529 non-null   float64
 3   Bedroom   7686 non-null   float64
 4   Bathroom  6490 non-null   float64
 5   Frontage  9671 non-null   int64  
 6   Floors    7654 non-null   float64
 7   Price     9671 non-null   object 
dtypes: float64(4), int64(1), object(3)
memory usage: 604.6+ KB


In [6]:
data_final.describe()

,Area,Bedroom,Bathroom,Frontage,Floors
count,9529.000000,7686.000000,6490.000000,9671.000000,7654.000000
mean,105.634157,3.677596,3.523421,0.397063,3.975568
std,1512.161797,3.556498,3.856034,0.489315,3.107054
min,2.000000,1.000000,1.000000,0.000000,1.000000
25%,43.000000,2.000000,2.000000,0.000000,2.000000
50%,60.000000,3.000000,3.000000,0.000000,4.000000
75%,85.000000,4.000000,4.000000,1.000000,5.000000
max,105119.000000,111.000000,111.000000,1.000000,93.000000


In [7]:
# Kiểm tra giá trị thiếu trong dữ liệu
print(data_final.isnull().sum())

ID             0
Location       0
Area         142
Bedroom     1985
Bathroom    3181
Frontage       0
Floors      2017
Price          0
dtype: int64


### 4. Preprocessing

    4.1 Chuyển đổi cột Frontage thành category, do đây là cột phân loại nhà mặt đường hay không

In [8]:
data_final["Frontage"]=data_final["Frontage"].astype("category")

    4.2 Thực hiện one-hot encoding chuyển cột "Location"

- Đếm số nhà bán của một khu vực.

In [9]:
location_stats = data_final.groupby('Location')['Location'].agg('count').sort_values(ascending=False)
print(location_stats.to_string())

Location
Gò Vấp, TP Hồ Chí Minh               803
Bình Thạnh, TP Hồ Chí Minh           655
Tân Bình, TP Hồ Chí Minh             615
Bình Tân, TP Hồ Chí Minh             560
Thủ Đức, TP Hồ Chí Minh              432
Tân Phú, TP Hồ Chí Minh              382
Quận 7, TP Hồ Chí Minh               324
Biên Hòa, Đồng Nai                   303
Hoàng Mai, Hà Nội                    301
Đống Đa, Hà Nội                      299
Phú Nhuận, TP Hồ Chí Minh            299
Thanh Xuân, Hà Nội                   284
Hà Đông, Hà Nội                      268
Cầu Giấy, Hà Nội                     262
Hai Bà Trưng, Hà Nội                 230
Quận 9, TP Hồ Chí Minh               228
Quận 10, TP Hồ Chí Minh              227
Long Biên, Hà Nội                    202
Quận 12, TP Hồ Chí Minh              194
Quận 8, TP Hồ Chí Minh               185
Tây Hồ, Hà Nội                       160
Huế, Thừa Thiên Huế                  159
Hải Châu, Đà Nẵng                    140
Thanh Trì, Hà Nội                    136
Tân Uyê

-   Những Quận Huyện của Tỉnh/Thành Phố có số nhà bán nhỏ hơn hoặc bằng 10

In [10]:
location_stats_less_than_10 = location_stats[location_stats <= 10]
print(location_stats_less_than_10.to_string())

Location
Ninh Kiều, Cần Thơ                   10
Cần Giuộc, Long An                    9
Bàu Bàng, Bình Dương                  8
Phan Thiết, Bình Thuận                8
Buôn Ma Thuột, Đắk Lắk                7
Tân An, Long An                       7
Thanh Hóa, Thanh Hóa                  7
Dĩ An, Bình Dương                     6
Ninh Bình, Ninh Bình                  6
Thới Lai, Cần Thơ                     6
Vị Thanh, Hậu Giang                   5
Vĩnh Yên, Vĩnh Phúc                   5
Thường Tín, Hà Nội                    5
Phú Quốc, Kiên Giang                  5
Hải An, Hải Phòng                     5
Phan Rang - Tháp Chàm, Ninh Thuận     4
Củ Chi, TP Hồ Chí Minh                4
Trảng Bom, Đồng Nai                   4
Long Thành, Đồng Nai                  3
Lào Cai, Lào Cai                      3
Bến Lức, Long An                      3
Châu Phú, An Giang                    3
Vũng Tàu, Bà Rịa Vũng Tàu             3
Yên Bái, Yên Bái                      3
Sóc Trăng, Sóc Trăng           

- Phân loại các Quận Huyện của Tỉnh/Thành Phố có số nhà bán lớn hơn 10 thành 1 nhóm và lưu các Tỉnh/Thành Phố của Quận Huyện đó vào Sets.

In [11]:
location_stats_not_less_than_10 = location_stats[location_stats > 10]
temp = location_stats_not_less_than_10.index.tolist()
for i in range(len(temp)):
    temp[i] = temp[i].split(", ")[1]
set_location_not_less_than_10 = set(temp)

-   Những Quận Huyện của Tỉnh/Thành Phố có số nhà bán <= 10 mà Tỉnh/Thành Phố nằm trong Sets thì ta sẽ nhóm Quận Huyện đó thành Ngoại Thành của Tỉnh/Thành Phố đó, còn nếu Tỉnh/Thành Phố đó không nằm trong Sets (nghĩa là Tỉnh/Thành Phố đó không có bất kỳ Quận Huyện nào có số nhà bán > 10) 
thì ta nhóm lại thành "Other". Bằng cách này khi ta One Hot Encoding, nó sẽ giúp ta ít dummy columns hơn.

In [12]:
def convert_location(x):
    temp = x.split(", ")[1]
    if temp in set_location_not_less_than_10:
        return "Ngoại thành của " + temp
    else:
        return "Other"

In [13]:
data_final.Location = data_final.Location.apply(lambda x: convert_location(x) if x in location_stats_less_than_10 else x)


- Thực hiện One Hot Encoding và xóa cột "other" để tránh dummy variable trap.

In [14]:
dummies = pd.get_dummies(data_final.Location)
data_final = pd.concat([data_final,dummies.drop('Other',axis='columns')],axis='columns')

### 5. Tìm kiếm outlier ở các cột feature

    5.1 Tách những observation có "Price" là "Thỏa thuận" ra file data_thoathuan.csv riêng

In [15]:
data_thoathuan = data_final[data_final['Price'] == "Thỏa thuận"]
data_thoathuan.to_csv('data_thoathuan.csv')

In [16]:
data_final = data_final[data_final['Price'] != "Thỏa thuận"]
data_final = data_final.reset_index(drop=True)
data_final

,ID,Location,Area,Bedroom,Bathroom,Frontage,Floors,Price,"An Dương, Hải Phòng","Ba Đình, Hà Nội",...,"Tân Uyên, Bình Dương","Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An"
0,268184,"Quận 12, TP Hồ Chí Minh",130.0,7.0,6.0,0,3.0,9.2 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
1,267895,"Quận 6, TP Hồ Chí Minh",189.0,NaN,NaN,1,NaN,24 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
2,268639,"Gò Vấp, TP Hồ Chí Minh",52.0,4.0,5.0,0,6.0,9.8 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
3,268638,"Gò Vấp, TP Hồ Chí Minh",105.0,5.0,2.0,1,4.0,8 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
4,268637,"Sơn Trà, Đà Nẵng",115.0,3.0,2.0,1,2.0,6.2 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8837,250238,"Hoàng Mai, Hà Nội",54.0,5.0,3.0,0,4.0,4.95 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
8838,250237,"Đống Đa, Hà Nội",50.0,4.0,NaN,0,5.0,7.5 tỷ,0,0,...,0,0,0,0,0,0,0,1,0,0
8839,250235,"Thủ Đức, TP Hồ Chí Minh",100.0,NaN,NaN,0,NaN,6 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
8840,249841,"Quận 12, TP Hồ Chí Minh",52.0,3.0,2.0,0,2.0,3.48 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0


    5.2 Xử lý  các cột Area, Bedroom, Bathroom ,Floors

- Hàm tìm outlier

In [17]:
def find_outliers_iqr(inp_data, coefficient):
    outliers=[]
    data=np.array(inp_data)
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    lower_bound = q1 - coefficient * iqr
    upper_bound = q3 + coefficient * iqr
    for i in range(0,len(data)):
        if (data[i] < lower_bound) or (data[i] > upper_bound):
            outliers.append(data[i])
    return outliers

- Thực hiện tìm kiếm những điểm outlier trên từng cột Area, Bedroom, Bathroom ,Floors của data và thay thế chúng bằng NaN

In [18]:
for columns in ["Area","Bedroom","Bathroom","Floors"]:
    data1 = data_final[~data_final[columns].isnull()][columns]
    temp_outlier=find_outliers_iqr(data1, 3)
    data_final[columns] = data_final[columns].apply(lambda x: np.nan if x in temp_outlier else x)


In [19]:
data_final

,ID,Location,Area,Bedroom,Bathroom,Frontage,Floors,Price,"An Dương, Hải Phòng","Ba Đình, Hà Nội",...,"Tân Uyên, Bình Dương","Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An"
0,268184,"Quận 12, TP Hồ Chí Minh",130.0,7.0,6.0,0,3.0,9.2 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
1,267895,"Quận 6, TP Hồ Chí Minh",189.0,NaN,NaN,1,NaN,24 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
2,268639,"Gò Vấp, TP Hồ Chí Minh",52.0,4.0,5.0,0,6.0,9.8 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
3,268638,"Gò Vấp, TP Hồ Chí Minh",105.0,5.0,2.0,1,4.0,8 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
4,268637,"Sơn Trà, Đà Nẵng",115.0,3.0,2.0,1,2.0,6.2 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8837,250238,"Hoàng Mai, Hà Nội",54.0,5.0,3.0,0,4.0,4.95 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
8838,250237,"Đống Đa, Hà Nội",50.0,4.0,NaN,0,5.0,7.5 tỷ,0,0,...,0,0,0,0,0,0,0,1,0,0
8839,250235,"Thủ Đức, TP Hồ Chí Minh",100.0,NaN,NaN,0,NaN,6 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0
8840,249841,"Quận 12, TP Hồ Chí Minh",52.0,3.0,2.0,0,2.0,3.48 tỷ,0,0,...,0,0,0,0,0,0,0,0,0,0


    5.3 Xử lý price

- Do trong cột "Price" ta có thể thấy được rằng cột có chưa 1 số data khác đơn vị như (triệu, nghìn). Cũng như để xử lý thuật toán ML thì các giá trị cần là số thực chứ không phải kiểu string , nên những đơn vị đo lường sẽ bị lược bỏ và điều chỉnh

In [20]:
data_final["Price"]

0        9.2 tỷ
1         24 tỷ
2        9.8 tỷ
3          8 tỷ
4        6.2 tỷ
         ...   
8837    4.95 tỷ
8838     7.5 tỷ
8839       6 tỷ
8840    3.48 tỷ
8841     1.1 tỷ
Name: Price, Length: 8842, dtype: object

- Hàm đổi đơn vị và bỏ đơn bị phía sau

In [21]:
def convert_price(price):
    if ('triệu' in price):
        converted_price = float(price.replace('triệu', ''))/1000
    elif 'nghìn' in price:
        converted_price = float(price.replace('nghìn', ''))/1000000
    else:
        converted_price = float(price.replace('tỷ', ''))
    return converted_price
    

In [22]:
data_final.Price = data_final.Price.apply(lambda x: convert_price(x))

- Thực hiện tìm outlier của cột "Price" và loại bỏ ra khỏi data chính, đồng thời lưu lại trên 1 file data_outlier.csv

In [23]:
price_data = data_final['Price']
price_outlier=find_outliers_iqr(price_data, 3)

In [24]:
data_outlier = data_final[data_final["Price"].isin(price_outlier)]
data_outlier.to_csv('data_outlier.csv')

In [25]:
data_final = data_final[~data_final["Price"].isin(price_outlier)]
data_final = data_final.reset_index(drop=True)
data_final

,ID,Location,Area,Bedroom,Bathroom,Frontage,Floors,Price,"An Dương, Hải Phòng","Ba Đình, Hà Nội",...,"Tân Uyên, Bình Dương","Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An"
0,268184,"Quận 12, TP Hồ Chí Minh",130.0,7.0,6.0,0,3.0,9.20,0,0,...,0,0,0,0,0,0,0,0,0,0
1,267895,"Quận 6, TP Hồ Chí Minh",189.0,NaN,NaN,1,NaN,24.00,0,0,...,0,0,0,0,0,0,0,0,0,0
2,268639,"Gò Vấp, TP Hồ Chí Minh",52.0,4.0,5.0,0,6.0,9.80,0,0,...,0,0,0,0,0,0,0,0,0,0
3,268638,"Gò Vấp, TP Hồ Chí Minh",105.0,5.0,2.0,1,4.0,8.00,0,0,...,0,0,0,0,0,0,0,0,0,0
4,268637,"Sơn Trà, Đà Nẵng",115.0,3.0,2.0,1,2.0,6.20,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,250238,"Hoàng Mai, Hà Nội",54.0,5.0,3.0,0,4.0,4.95,0,0,...,0,0,0,0,0,0,0,0,0,0
7901,250237,"Đống Đa, Hà Nội",50.0,4.0,NaN,0,5.0,7.50,0,0,...,0,0,0,0,0,0,0,1,0,0
7902,250235,"Thủ Đức, TP Hồ Chí Minh",100.0,NaN,NaN,0,NaN,6.00,0,0,...,0,0,0,0,0,0,0,0,0,0
7903,249841,"Quận 12, TP Hồ Chí Minh",52.0,3.0,2.0,0,2.0,3.48,0,0,...,0,0,0,0,0,0,0,0,0,0


- Thực hiện xóa cột "Location" sau khi đã thực hiện feature engineering để tạo ra one-hot thay thế cho cột "Location"

In [26]:
del data_final["Location"]
data_final

,ID,Area,Bedroom,Bathroom,Frontage,Floors,Price,"An Dương, Hải Phòng","Ba Đình, Hà Nội","Biên Hòa, Đồng Nai",...,"Tân Uyên, Bình Dương","Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An"
0,268184,130.0,7.0,6.0,0,3.0,9.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,267895,189.0,NaN,NaN,1,NaN,24.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,268639,52.0,4.0,5.0,0,6.0,9.80,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,268638,105.0,5.0,2.0,1,4.0,8.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,268637,115.0,3.0,2.0,1,2.0,6.20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,250238,54.0,5.0,3.0,0,4.0,4.95,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7901,250237,50.0,4.0,NaN,0,5.0,7.50,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7902,250235,100.0,NaN,NaN,0,NaN,6.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7903,249841,52.0,3.0,2.0,0,2.0,3.48,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Filling missing data

In [27]:
del data_final["ID"]
data_final

,Area,Bedroom,Bathroom,Frontage,Floors,Price,"An Dương, Hải Phòng","Ba Đình, Hà Nội","Biên Hòa, Đồng Nai","Bình Chánh, TP Hồ Chí Minh",...,"Tân Uyên, Bình Dương","Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An"
0,130.0,7.0,6.0,0,3.0,9.20,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,189.0,NaN,NaN,1,NaN,24.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,52.0,4.0,5.0,0,6.0,9.80,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,105.0,5.0,2.0,1,4.0,8.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,115.0,3.0,2.0,1,2.0,6.20,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,54.0,5.0,3.0,0,4.0,4.95,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7901,50.0,4.0,NaN,0,5.0,7.50,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7902,100.0,NaN,NaN,0,NaN,6.00,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7903,52.0,3.0,2.0,0,2.0,3.48,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


- Sử dụng MICE để fill data

In [28]:
X_train=data_final.drop(columns='Price')
y_train=data_final["Price"]

- Sử dụng IterativeImputer của thư viện sklearn để fit với data từ phần X_train, khi này ta sẽ có được data_filled

In [29]:
mice_imputer = IterativeImputer()
X_train_filled = pd.DataFrame(mice_imputer.fit_transform(X_train), columns=X_train.columns)
data_filled = pd.concat([X_train_filled,y_train],axis=1)
data_filled

,Area,Bedroom,Bathroom,Frontage,Floors,"An Dương, Hải Phòng","Ba Đình, Hà Nội","Biên Hòa, Đồng Nai","Bình Chánh, TP Hồ Chí Minh","Bình Thạnh, TP Hồ Chí Minh",...,"Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An",Price
0,130.0,7.000000,6.000000,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.20
1,189.0,5.040290,4.504500,1.0,4.203619,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.00
2,52.0,4.000000,5.000000,0.0,6.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.80
3,105.0,5.000000,2.000000,1.0,4.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.00
4,115.0,3.000000,2.000000,1.0,2.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7900,54.0,5.000000,3.000000,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.95
7901,50.0,4.000000,3.876864,0.0,5.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7.50
7902,100.0,3.848917,3.652690,0.0,3.252786,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.00
7903,52.0,3.000000,2.000000,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.48


- Tận dụng data_thoathoan và data_outlier ở trên

In [30]:
a=pd.read_csv("data_outlier.csv")
b=pd.read_csv("data_thoathuan.csv")

In [31]:
data_thoathuan_outlier=pd.concat([a,b],axis=0).reset_index()
data_thoathuan_outlier=data_thoathuan_outlier.drop(["index","Unnamed: 0","ID","Location"], axis=1)
data_thoathuan_outlier

,Area,Bedroom,Bathroom,Frontage,Floors,Price,"An Dương, Hải Phòng","Ba Đình, Hà Nội","Biên Hòa, Đồng Nai","Bình Chánh, TP Hồ Chí Minh",...,"Tân Uyên, Bình Dương","Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An"
0,56.0,2.0,2.0,1,2.0,4700000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,106.0,3.0,2.0,0,2.0,4900000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40.0,2.0,2.0,0,3.0,4900000000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,4.0,3.0,0,3.0,3260.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,40.0,3.0,2.0,0,NaN,3000000.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761,63.0,4.0,4.0,0,3.0,Thỏa thuận,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1762,105.0,3.0,2.0,1,3.0,Thỏa thuận,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1763,112.0,3.0,2.0,1,1.0,Thỏa thuận,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1764,70.0,NaN,NaN,0,8.0,Thỏa thuận,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


- Thực hiện fill bằng MICE cho tập data này với "mice_imputer = IterativeImputer()" đã được train ở trên, ta được 1 tập data với lượng input đầy đủ, tuy nhiên Label của data này là outlier hay mang giá trị "Thỏa thuận" nên ta sẽ xóa bỏ cột "Price" của chúng

In [32]:
data_thoathuan_outlier["Price"]=np.nan
X_thoathuan_outlier=data_thoathuan_outlier.drop(columns='Price')
y_thoathuan_outlier=data_thoathuan_outlier["Price"]

In [33]:
X_thoathuan_outlier_filled=pd.DataFrame(mice_imputer.transform(X_thoathuan_outlier), columns=X_thoathuan_outlier.columns)
data_thoathuan_outlier_filled = pd.concat([X_thoathuan_outlier_filled,y_thoathuan_outlier],axis=1)
data_thoathuan_outlier_filled

,Area,Bedroom,Bathroom,Frontage,Floors,"An Dương, Hải Phòng","Ba Đình, Hà Nội","Biên Hòa, Đồng Nai","Bình Chánh, TP Hồ Chí Minh","Bình Thạnh, TP Hồ Chí Minh",...,"Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An",Price
0,56.000000,2.000000,2.000000,1.0,2.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,106.000000,3.000000,2.000000,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,40.000000,2.000000,2.000000,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,51.804613,4.000000,3.000000,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,40.000000,3.000000,2.000000,0.0,2.312442,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761,63.000000,4.000000,4.000000,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
1762,105.000000,3.000000,2.000000,1.0,3.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1763,112.000000,3.000000,2.000000,1.0,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1764,70.000000,5.289737,5.411861,0.0,8.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN


In [34]:
#BỎ
data_filled=data_filled[data_filled.Price <= 10].reset_index()
del data_filled["index"]
data_filled

,Area,Bedroom,Bathroom,Frontage,Floors,"An Dương, Hải Phòng","Ba Đình, Hà Nội","Biên Hòa, Đồng Nai","Bình Chánh, TP Hồ Chí Minh","Bình Thạnh, TP Hồ Chí Minh",...,"Tây Hồ, Hà Nội","Từ Sơn, Bắc Ninh","Vĩnh Cửu, Đồng Nai","Đà Lạt, Lâm Đồng","Đông Anh, Hà Nội","Đất Đỏ, Bà Rịa Vũng Tàu","Đống Đa, Hà Nội","Đồng Hới, Quảng Bình","Đức Hòa, Long An",Price
0,130.0,7.000000,6.000000,0.0,3.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.20
1,52.0,4.000000,5.000000,0.0,6.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.80
2,105.0,5.000000,2.000000,1.0,4.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.00
3,115.0,3.000000,2.000000,1.0,2.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.20
4,48.0,4.000000,5.000000,0.0,5.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6748,54.0,5.000000,3.000000,0.0,4.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.95
6749,50.0,4.000000,3.876864,0.0,5.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7.50
6750,100.0,3.848917,3.652690,0.0,3.252786,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.00
6751,52.0,3.000000,2.000000,0.0,2.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.48


- Lưu trữ 2 tập data ra 2 file csv

In [35]:
data_filled.to_csv("data_train.csv")
data_thoathuan_outlier_filled.to_csv("data_thoathuan_outlier.csv")